In [1]:
from keras.datasets import fashion_mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import BatchNormalization, Dropout
from keras.optimizers import SGD
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
# load data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 4s 1us/step


In [3]:
# inspect shape, max/min
print('x_train.shape={}, y_train.shape={}'.format(x_train.shape, y_train.shape))
print('x_test.shape={}, y_test.shape={}'.format(x_test.shape, y_test.shape))
print('x_train[0].max()={}, x_train[0].min()={}'.format(x_train[0].max(), x_train[0].min()))
print('y_train.max()={}, y_train.min()={}'.format(y_train.max(), y_train.min()))

x_train.shape=(60000, 28, 28), y_train.shape=(60000,)
x_test.shape=(10000, 28, 28), y_test.shape=(10000,)
x_train[0].max()=255, x_train[0].min()=0
y_train.max()=9, y_train.min()=0


In [4]:
# add 1 dim to x_train and x_test
x_train_expand = np.expand_dims(x_train, axis=-1)
x_test_expand = np.expand_dims(x_test, axis=-1)

# encode y_train and y_test
encoder = OneHotEncoder(sparse=False)
y_train_encodded = encoder.fit_transform(y_train.reshape(len(y_train), 1))
y_test_encodded = encoder.transform(y_test.reshape(len(y_test), 1))

# inspect shape
print('x_train_expand.shape={}, y_train_encodded.shape={}'.format( \
    x_train_expand.shape, y_train_encodded.shape))
print('x_test_expand.shape={}, y_test_encodded.shape={}'.format( \
    x_test_expand.shape, y_test_encodded.shape))
print(y_train_encodded[0])

x_train_expand.shape=(60000, 28, 28, 1), y_train_encodded.shape=(60000, 10)
x_test_expand.shape=(10000, 28, 28, 1), y_test_encodded.shape=(10000, 10)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]


In [11]:
# Define CNN
class CNN(object):
    def __init__(self):
        self.model = self.build_model()
        
        # Image data augmentation
        self.train_datagen = ImageDataGenerator(
                    rotation_range=0,
                    rescale=1./255,
                    horizontal_flip=True,
                    fill_mode='nearest')

        self.valid_datagen = ImageDataGenerator(rescale=1./255)
        
    
    def build_model(self, kernel_size=(3, 3), strides=1, pool_size=(2, 2), activation='relu'):
        model = Sequential()
        
        ## Layer 0:
        # input shape = (None, 28, 28, 1), output shape=(None, 14, 14, 32)
        model.add(Conv2D(32, kernel_size=kernel_size, 
                         strides=strides, padding='same', activation=activation,
                        input_shape=(28, 28, 1)))
        model.add(Conv2D(32, kernel_size=kernel_size, 
                         strides=strides, padding='same', activation=activation))
        model.add(MaxPooling2D(pool_size=pool_size))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        
        ## Layer 1:
        # output shape=(None, 7, 7, 64)
        model.add(Conv2D(64, kernel_size=kernel_size,
                        strides=strides, padding='same', activation=activation))
        model.add(Conv2D(64, kernel_size=kernel_size,
                        strides=strides, padding='same', activation=activation))
        model.add(MaxPooling2D(pool_size=pool_size))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        
        ## Layer 2:
        # output shape = (None, 5, 5, 128) => (7 - 3 + 2*0) / 1 + 1 = 5
        model.add(Conv2D(128, kernel_size=kernel_size,
                        strides=strides, padding='valid', activation=activation))
        
        # output shape = (None, 3, 3, 128) => (5 - 3 + 2*0) / 1 + 1 = 3
        model.add(Conv2D(128, kernel_size=kernel_size,
                        strides=strides, padding='valid', activation=activation))
        
        # output shape = (None, 3, 3, 128)
        model.add(MaxPooling2D(pool_size=pool_size))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        
        ## Layer 3
        # output shape = (None, 128)
        model.add(GlobalAveragePooling2D())
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        
        ## Layer 4
        # output shape = (None, 128)
        model.add(Dense(128, activation=activation))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        
        ## Layer 5
        # Output shape = (None, 64)
        model.add(Dense(64, activation=activation))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        
        ## Layer 6
        # output shape = (None, 10)
        model.add(Dense(10, activation='softmax'))
        
        # define optimizer
        sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
        
        return model
    
    
    def fit(self, x_train, y_train, x_valid, y_valid, epochs, batch_size=32, batch_size_valid=32):
        

        # generators
        train_generator = self.train_datagen.flow(x_train, y_train,batch_size=batch_size)
        
        valid_generator = self.valid_datagen.flow(x_valid, y_valid, batch_size=batch_size_valid)

        self.model.fit_generator(
                    train_generator,
                    epochs=epochs,
                    steps_per_epoch=len(x_train) / batch_size * 2,
                    validation_data=valid_generator,
                    validation_steps=(len(x_valid)/ batch_size_valid))

In [12]:
cnn = CNN()
cnn.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 14, 14, 32)        128       
_________________________________________________________________
dropout_13 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 14, 14, 64)        36928     
__________

In [13]:
epochs = 10
cnn.fit(x_train_expand, y_train_encodded, x_test_expand, y_test_encodded, epochs)

Epoch 1/10
3750/3750 [==============================] - 91s 24ms/step - loss: 0.7154 - acc: 0.7371 - val_loss: 0.5602 - val_acc: 0.8038
Epoch 2/10
3750/3750 [==============================] - 90s 24ms/step - loss: 0.4714 - acc: 0.8333 - val_loss: 0.3292 - val_acc: 0.8775
Epoch 3/10
3750/3750 [==============================] - 91s 24ms/step - loss: 0.3850 - acc: 0.8659 - val_loss: 0.2886 - val_acc: 0.8967
Epoch 4/10
3750/3750 [==============================] - 90s 24ms/step - loss: 0.3453 - acc: 0.8798 - val_loss: 0.2765 - val_acc: 0.8973
Epoch 5/10
3750/3750 [==============================] - 90s 24ms/step - loss: 0.3197 - acc: 0.8891 - val_loss: 0.2839 - val_acc: 0.8944
Epoch 6/10
3750/3750 [==============================] - 91s 24ms/step - loss: 0.3021 - acc: 0.8952 - val_loss: 0.2525 - val_acc: 0.9097
Epoch 7/10
3750/3750 [==============================] - 91s 24ms/step - loss: 0.2883 - acc: 0.9003 - val_loss: 0.2523 - val_acc: 0.9097
Epoch 8/10
3750/3750 [==========================